In [9]:
import pandas as pd

In [10]:
data = pd.read_csv("../../data/curated/API_re_clean.csv", index_col=0)
SA2 = pd.read_csv("../../data/raw/SA2_2016_AUST.csv")

In [11]:
suburb = data['suburb'].unique().tolist()
# print(suburb)
# print("length:", len(suburb))

In [12]:
from operator import concat


COL = ["SA2_MAINCODE_2016", "SA2_5DIGITCODE_2016", "SA2_NAME_2016", "STATE_CODE_2016"]
SA2 = SA2[COL]

# Slice the SA2 for Victoria
SA2 = SA2.loc[SA2["STATE_CODE_2016"] == 2]

# Make all lettters to uppercase
SA2['SA2_NAME_2016'] = SA2['SA2_NAME_2016'].str.upper()

# Categorize the SA2_NAME into 'single', 'multiple' and 'contained (.vic)'
valid_pattern = '^[ A-Za-z\(\)\.]+$'
vic_pattern = '[ A-Za-z\-]*[ ]\(VIC\.\)'
multiple_pattern = '^[ A-Za-z\(\)\-]*-[ A-Za-z\(\)\-]*'
SA2["SINGLE_NAME"] = SA2['SA2_NAME_2016'].str.match(pat = valid_pattern)
SA2["VIC_NAME"] = SA2['SA2_NAME_2016'].str.match(pat = vic_pattern)
SA2["MULTIPLE_NAME"] = SA2['SA2_NAME_2016'].str.match(pat = multiple_pattern)

# Put single name as a valid name
SA2['VALID_NAME'] = SA2["SINGLE_NAME"]

# Remove (VIC.) on the SA2_NAME
vic_removed = SA2.loc[SA2["VIC_NAME"] == True, "SA2_NAME_2016"].str.extract(r'([ A-Za-z\(\)\-]*)[ ]\(VIC\.\)', expand=False)
SA2.loc[SA2["VIC_NAME"] == True, "SA2_NAME_2016"] = vic_removed

# Remove multiple SA2_NAME
multiple_random = SA2.copy()
mr = multiple_random.loc[multiple_random["MULTIPLE_NAME"] == True, "SA2_NAME_2016"].str.extract(r"([ A-Za-z]*)[ ]-[ A-Za-z\(\)\-]*", expand=False)
multiple_random.loc[multiple_random["MULTIPLE_NAME"] == True, "SA2_NAME_2016"] = mr
multiple_random = multiple_random.loc[multiple_random["MULTIPLE_NAME"]==True]
multiple_random = multiple_random.drop_duplicates(subset=['SA2_NAME_2016'])
multiple_random.dropna()

multiple_random2 = SA2.copy()
mr = multiple_random2.loc[multiple_random2["MULTIPLE_NAME"] == True, "SA2_NAME_2016"].str.extract(r"([A-Za-z]*)[ ]-[ A-Za-z\(\)\-]*", expand=False)
multiple_random2.loc[multiple_random2["MULTIPLE_NAME"] == True, "SA2_NAME_2016"] = mr
multiple_random2 = multiple_random2.loc[multiple_random2["MULTIPLE_NAME"]==True]
multiple_random2 = multiple_random2.drop_duplicates(subset=['SA2_NAME_2016'])
multiple_random2.dropna()

multiple_removed = SA2.loc[SA2["MULTIPLE_NAME"] == True, "SA2_NAME_2016"].str.split(r"[ ]-[ ]").apply(lambda x: ' '.join(x))
SA2.loc[SA2["MULTIPLE_NAME"] == True, "SA2_NAME_2016"] = multiple_removed

SA2 = pd.concat([SA2, multiple_random, multiple_random2], ignore_index=True)
SA2 = SA2.drop_duplicates(subset=['SA2_NAME_2016'])
SA2 = SA2.dropna()
SA2.to_csv("../../data/curated/edited_SA2.csv")
SA2

,SA2_MAINCODE_2016,SA2_5DIGITCODE_2016,SA2_NAME_2016,STATE_CODE_2016,SINGLE_NAME,VIC_NAME,MULTIPLE_NAME,VALID_NAME
0,201011001,21001,ALFREDTON,2,True,False,False,True
1,201011002,21002,BALLARAT,2,True,False,False,True
2,201011003,21003,BALLARAT NORTH,2,False,False,True,False
3,201011004,21004,BALLARAT SOUTH,2,False,False,True,False
4,201011005,21005,BUNINYONG,2,True,False,False,True
...,...,...,...,...,...,...,...,...
659,212021455,21455,WARREN,2,False,False,True,False
664,212051321,21321,WAVERLEY,2,False,False,True,False
667,213011334,21334,ALBANS,2,False,False,True,False
671,213051361,21361,CROSSING,2,False,False,True,False


In [6]:
import re

In [8]:
data = data.merge(SA2, how="left", left_on="suburb", right_on="SA2_NAME_2016")
# data = data.dropna()
data.to_csv("../../data/curated/API_clean_price_with_SA2.csv")